1. train.csv : 학습 데이터
2. test.csv : 테스트 데이터
- id : 샘플 아이디
- Store : 쇼핑몰 지점
- Date : 주 단위(Weekly) 날짜
- 2012-10-05    45
- 2012-10-12    45
- 2012-10-19    45
- 2012-10-26    45
- Temperature : 해당 쇼핑몰 주변 기온

- Fuel_Price : 해당 쇼핑몰 주변 연료 가격
- Promotion 1~5 : 해당 쇼핑몰의 비식별화된 프로모션 정보
- Unemployment : 해당 쇼핑몰 지역의 실업률
- IsHoliday : 해당 기간의 공휴일 포함 여부



- id 칼럼은 각 행의 고유한 id를 나타냅니다. 총 6,255 개의 훈련용 데이터가 있네요.
-  Store는 쇼핑몰 지점 정보입니다. 1에서 45 까지의 지점이 있는듯 합니다.
-  Date는 주 단위(Weekly) 날짜입니다.2010년 2월 5일부터 2012년 9월 28일까지 있는 것 같네요.
-  Temperature는 쇼핑몰 주변 기온입니다. 아무래도 화씨(F)로 표현된 것 같습니다.
-  Fuel_Price는 쇼핑몰 주변 연료 가격입니다. 2년 사이에 1.6배나 올랐네요.
-  Promotion은 쇼핑몰의 비식별화된 프로모션 정보입니다. 잘 모르겠지만 nan 값들이 많이 보이네요.
-  Unemployment는 실업률 정보입니다. 아무래도 실업률이 높아지면 소비가 위축되지 않을까 싶네요.
-  IsHoliday는 공휴일 정보입니다. 휴일이면 쇼핑하러 갈 시간이 더 많아지지 않을까요?
- Weekly_Sales는 주간 매출액으로 이번 대회의 목표 예측값입니다. 지점별로 차이가 큰 모습을 보이네요.

In [98]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import joblib
import os
import time
import glob

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from itertools import permutations, combinations
import warnings

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor, plot_tree, plot_importance
from lightgbm import LGBMRegressor
#from catboost import CatBoostClassifier, Pool, CatBoostRegressor

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostRegressor,  HistGradientBoostingRegressor, StackingRegressor, RandomForestRegressor

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

data = pd.concat([train, test], ignore_index = True)
print('train shape:', train.shape, '  ', 'test shape: ', test.shape)
train.head()

train shape: (6255, 13)    test shape:  (180, 12)


,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68


#### 날짜 변환

In [99]:
# n주차 만들기
from datetime import datetime, timedelta

def change_df(df):
    df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
    
    df['Date'] = df['Date'].astype('str')
    return df

train = change_df(train)
test = change_df(test)

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day=1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days=6-firstday.weekday())
    return (target - origin).days // 7 + 1

def week(df):
    a = []
    for i in df['Date'].values:
        yi = int(i[:4])
        mi = int(i[5:7])
        di = int(i[8:])
        a.append(get_week_no(yi, mi, di))

    df['week'] = a
    return df

def change_week(df):
    df = week(df)
    df['Date1'] = df['Date'].str[:7]
    t = df[df['week']==0]
    t = t.drop_duplicates(['Date1'])
    df.loc[df["Date1"].isin(t["Date1"]), "week"] = df.loc[df["Date1"].isin(t["Date1"]), "week"] + 1
    return df

train = change_week(train)
test = change_week(test)

def date_conv(df):
    df['Date'] = pd.to_datetime(df['Date'])
    
    df['year'] = df['Date'].dt.year.apply(lambda x: int(x))
    df['month'] =df['Date'].dt.month.apply(lambda x: int(x))
    df['day'] =df['Date'].dt.day.apply(lambda x: int(x))
    df['month_day'] =df['Date'].dt.strftime('%m-%d')
    df['weekday'] = df['Date'].dt.weekday
    return df
    
train = date_conv(train)
test = date_conv(test)

# date 삭제
#train = train.drop(columns=['Date', 'month_day', 'Date1'])
#test = test.drop(columns=['Date', 'month_day', 'Date1'])
train.shape

(6255, 20)

In [94]:
train


,id,Store,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,week,year,month,day,weekday
0,1,1,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90,1,2010,2,5,4
1,2,1,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44,2,2010,2,12,4
2,3,1,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17,3,2010,2,19,4
3,4,1,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59,4,2010,2,26,4
4,5,1,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68,1,2010,3,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251,45,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,False,734297.87,5,2012,8,31,4
6251,6252,45,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66,1,2012,9,7,4
6252,6253,45,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27,2,2012,9,14,4
6253,6254,45,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20,3,2012,9,21,4


In [42]:
#plt.figure(figsize = (20, 12))
#sns.lineplot(x = 'week', y = 'Weekly_Sales', data = train, hue = 'year')
#plt.show()

In [43]:
#import pandas as pd

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

#### 결측치 처리

In [104]:
a = train[['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5', 'Store', 'Date']].loc[90:100]
a


,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Store,Date
90,NaN,NaN,NaN,NaN,NaN,1,2011-10-28
91,NaN,NaN,NaN,NaN,NaN,1,2011-11-04
92,10382.90,6115.67,215.07,2406.62,6551.42,1,2011-11-11
93,6074.12,254.39,51.98,427.39,5988.57,1,2011-11-18
94,410.31,98.00,55805.51,8.00,554.92,1,2011-11-25
95,5629.51,68.00,1398.11,2084.64,20475.32,1,2011-12-02
96,4640.65,19.00,105.02,3639.42,14461.82,1,2011-12-09
97,5011.32,67.00,347.37,225.79,4011.37,1,2011-12-16
98,2725.36,40.48,634.70,24.90,2739.43,1,2011-12-23
99,5762.10,46011.38,260.36,983.65,4735.78,1,2011-12-30


In [88]:
def holiday(series):
    if series == True:
        number = 1
    else:
        number = 0       
    return number

train['IsHoliday'] = train['IsHoliday'].apply(holiday)
test['IsHoliday'] = test['IsHoliday'].apply(holiday)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 

train_copy = train.copy()
test_copy = test.copy()

def null(df): 
    # 결측치(iterative)
    df = IterativeImputer(random_state=2021).fit_transform(df)
    df = pd.DataFrame(df)
    if df.shape[1] == train_copy.shape[1]:
        df.columns = train_copy.columns
    else:
        df.columns = test_copy.columns 
    return df

train = null(train)
test = null(test)

from sklearn.preprocessing import StandardScaler

def scaler(df):
    ss = StandardScaler()
    scaled = ss.fit_transform(df[['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']])
    df[['Scaled_Promotion1','Scaled_Promotion2', 'Scaled_Promotion3','Scaled_Promotion4', 'Scaled_Promotion5']] = scaled
    df = df.drop(columns=['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5'])
    return df

train = scaler(train)
test = scaler(test)
train

,id,Store,Temperature,Fuel_Price,Unemployment,IsHoliday,Weekly_Sales,week,year,month,day,weekday,Scaled_Promotion1,Scaled_Promotion2,Scaled_Promotion3,Scaled_Promotion4,Scaled_Promotion5
0,1.0,1.0,42.31,2.572,8.106,0.0,1643690.90,1.0,2010.0,2.0,5.0,4.0,0.599723,0.197527,0.311419,0.497698,0.464315
1,2.0,1.0,38.51,2.548,8.106,1.0,1641957.44,2.0,2010.0,2.0,12.0,4.0,0.598296,0.196964,0.310399,0.496582,0.462912
2,3.0,1.0,39.93,2.514,8.106,0.0,1611968.17,3.0,2010.0,2.0,19.0,4.0,0.578676,0.187274,0.291425,0.481093,0.438704
3,4.0,1.0,46.63,2.561,8.106,0.0,1409727.59,4.0,2010.0,2.0,26.0,4.0,0.448027,0.121943,0.163101,0.378377,0.275494
4,5.0,1.0,46.50,2.625,8.106,0.0,1554806.68,1.0,2010.0,3.0,5.0,4.0,0.542156,0.168809,0.255074,0.452408,0.392607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251.0,45.0,75.09,3.867,8.684,0.0,734297.87,5.0,2012.0,8.0,31.0,4.0,3.195845,-0.652682,-0.213502,1.128841,-0.120387
6251,6252.0,45.0,75.70,3.911,8.684,1.0,766512.66,1.0,2012.0,9.0,7.0,4.0,0.761856,-0.651263,-0.220780,-0.248179,-0.633372
6252,6253.0,45.0,67.87,3.948,8.684,0.0,702238.27,2.0,2012.0,9.0,14.0,4.0,0.835839,-0.118019,-0.229508,0.172139,0.217851
6253,6254.0,45.0,65.32,4.038,8.684,0.0,723086.20,3.0,2012.0,9.0,21.0,4.0,0.265628,-0.634680,-0.218864,-0.108263,1.118947


#### 변수 생성

In [46]:
from itertools import permutations, combinations
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

target_columns = ['Temperature', 'Fuel_Price', 'week', 'month', 'day', 'year', 'Scaled_Promotion1', 'Scaled_Promotion2', 'Scaled_Promotion3', 'Scaled_Promotion4', 'Scaled_Promotion5']

combis = list(combinations(target_columns, 2))

for com in combis:
    train['{}*{}'.format(com[0], com[1])] = train['{}'.format(com[0])] * train['{}'.format(com[1])]
    test['{}*{}'.format(com[0], com[1])] = test['{}'.format(com[0])] * test['{}'.format(com[1])]
    train['{}+{}'.format(com[0], com[1])] = train['{}'.format(com[0])] + train['{}'.format(com[1])]
    test['{}+{}'.format(com[0], com[1])] = test['{}'.format(com[0])] + test['{}'.format(com[1])]
    train['{}/{}'.format(com[0], com[1])] = train['{}'.format(com[0])] / train['{}'.format(com[1])]
    test['{}/{}'.format(com[0], com[1])] = test['{}'.format(com[0])] / test['{}'.format(com[1])]
    
print(train.shape)
# 생성후 결측치 확인
print(train.isnull().sum().sort_values())

(6255, 182)
id                                     0
month*Scaled_Promotion5                0
month+Scaled_Promotion5                0
month/Scaled_Promotion5                0
day*year                               0
                                      ..
Fuel_Price+Scaled_Promotion2           0
Fuel_Price/Scaled_Promotion2           0
Fuel_Price*Scaled_Promotion3           0
Fuel_Price/Scaled_Promotion3           0
Scaled_Promotion4/Scaled_Promotion5    0
Length: 182, dtype: int64


In [128]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Columns: 182 entries, id to Scaled_Promotion4/Scaled_Promotion5
dtypes: float64(182)
memory usage: 8.7 MB


#### 변수 선택

In [49]:
# good
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

target = train['Weekly_Sales']
train1 = train.drop(['Weekly_Sales'], axis=1)

model = ExtraTreesRegressor(n_estimators=50).fit(train1, target)
selector = SelectFromModel(model, threshold = 'median', prefit=True)

train1 = train1[train1.columns[selector.get_support(indices=True)]]
train# 1095 rows × 116 columns
train2 = train[['month']]
train = pd.concat([train1, train2, target], axis=1)
train

test1 = test[test.columns[selector.get_support(indices=True)]]
test2 = test[['month']]
test = pd.concat([test1, test2], axis=1)
test # 365 rows × 116 columns

,id,Store,Unemployment,year,Scaled_Promotion1,Scaled_Promotion2,Scaled_Promotion3,Scaled_Promotion4,Scaled_Promotion5,Temperature*Scaled_Promotion1,...,Scaled_Promotion2+Scaled_Promotion5,Scaled_Promotion2/Scaled_Promotion5,Scaled_Promotion3*Scaled_Promotion4,Scaled_Promotion3+Scaled_Promotion4,Scaled_Promotion3*Scaled_Promotion5,Scaled_Promotion3+Scaled_Promotion5,Scaled_Promotion3/Scaled_Promotion5,Scaled_Promotion4*Scaled_Promotion5,Scaled_Promotion4+Scaled_Promotion5,month
0,1.0,1.0,6.573,2012.0,1.575038,1.046505,-0.259856,1.676705,0.060872,107.968882,...,1.107376,17.191966,-0.435703,1.416848,-0.015818,-0.198985,-4.268919,0.102064,1.737576,5.0
1,2.0,1.0,6.573,2012.0,-0.396104,0.234571,-0.480285,-0.326720,0.862308,-24.950599,...,1.096879,0.272027,0.156919,-0.807005,-0.414153,0.382023,-0.556976,-0.281733,0.535588,12.0
2,3.0,1.0,6.573,2012.0,-0.769774,-0.674671,-0.549618,-0.673647,-0.396683,-52.321548,...,-1.071354,1.700782,0.370249,-1.223265,0.218024,-0.946301,1.385536,0.267224,-1.070330,10.0
3,4.0,1.0,6.573,2012.0,-0.231724,-0.857317,-0.526289,-0.024519,-0.747818,-16.026001,...,-1.605135,1.146425,0.012904,-0.550808,0.393568,-1.274107,0.703767,0.018336,-0.772337,10.0
4,5.0,2.0,6.170,2012.0,0.903880,0.737159,-0.438205,1.284627,0.140050,63.515642,...,0.877209,5.263548,-0.562930,0.846422,-0.061371,-0.298155,-3.128922,0.179912,1.424677,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,176.0,44.0,5.217,2012.0,-1.069697,-1.406458,-0.635740,-1.511966,-1.043010,-50.243671,...,-2.449468,1.348461,0.961218,-2.147706,0.663083,-1.678750,0.609525,1.576995,-2.554975,10.0
176,177.0,45.0,8.667,2012.0,0.577856,0.200697,-0.246775,0.770367,-0.387220,37.497066,...,-0.186523,-0.518303,-0.190107,0.523592,0.095556,-0.633995,0.637298,-0.298302,0.383147,5.0
177,178.0,45.0,8.667,2012.0,-0.438838,-0.147062,-0.485082,-0.328740,0.187830,-23.903528,...,0.040768,-0.782954,0.159466,-0.813822,-0.091113,-0.297252,-2.582557,-0.061747,-0.140910,12.0
178,179.0,45.0,8.667,2012.0,-0.423133,-0.630360,-0.587774,-0.436112,-0.666821,-23.894320,...,-1.297181,0.945321,0.256335,-1.023885,0.391940,-1.254595,0.881457,0.290808,-1.102933,10.0


In [111]:
# 4. 순환하면서 변수 제거/후진소거법 (https://process-mining.tistory.com/138)
from sklearn.feature_selection import RFECV 
from sklearn.tree import DecisionTreeClassifier

target = train['Weekly_Sales']
train1 = train.drop(['Weekly_Sales'], axis=1)
model = LinearRegression()

selector = RFECV(model, step=1, scoring = 'neg_mean_squared_error')
selector.fit(train1, target)
selector.n_features_ # 117

train2 = selector.transform(train1)
selected_features = train1.columns[np.where(selector.ranking_==1)]
train = pd.DataFrame(train2, columns = selected_features)
train = pd.concat([train, target], axis=1)
train


,id,Store,Temperature,Fuel_Price,Unemployment,IsHoliday,week,year,month,day,...,Scaled_Promotion2+Scaled_Promotion5,Scaled_Promotion2/Scaled_Promotion5,Scaled_Promotion3*Scaled_Promotion4,Scaled_Promotion3+Scaled_Promotion4,Scaled_Promotion3*Scaled_Promotion5,Scaled_Promotion3+Scaled_Promotion5,Scaled_Promotion3/Scaled_Promotion5,Scaled_Promotion4*Scaled_Promotion5,Scaled_Promotion4+Scaled_Promotion5,Weekly_Sales
0,1.0,1.0,42.31,2.572,8.106,0.0,1.0,2010.0,5.0,2.0,...,0.661884,0.425376,0.155012,0.809191,0.144600,0.775756,0.670602,0.231153,0.962149,1643690.90
1,2.0,1.0,38.51,2.548,8.106,1.0,1.0,2010.0,12.0,2.0,...,0.659929,0.425442,0.154199,0.807214,0.143675,0.773301,0.670325,0.230037,0.959842,1641957.44
2,3.0,1.0,39.93,2.514,8.106,0.0,3.0,2010.0,2.0,19.0,...,0.626016,0.426840,0.140199,0.772507,0.127861,0.730168,0.664227,0.211071,0.919825,1611968.17
3,4.0,1.0,46.63,2.561,8.106,0.0,4.0,2010.0,2.0,26.0,...,0.397477,0.442564,0.061711,0.541458,0.044940,0.438637,0.591948,0.104250,0.653891,1409727.59
4,5.0,1.0,46.50,2.625,8.106,0.0,1.0,2010.0,5.0,3.0,...,0.561459,0.429921,0.115409,0.707543,0.100147,0.647704,0.649568,0.177670,0.845140,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251.0,45.0,75.09,3.867,8.684,0.0,5.0,2012.0,8.0,31.0,...,-0.773070,5.421493,-0.241010,0.915339,0.025703,-0.333890,1.773454,-0.135899,1.008453,734297.87
6251,6252.0,45.0,75.70,3.911,8.684,1.0,2.0,2012.0,7.0,9.0,...,-1.284636,1.028246,0.054793,-0.468958,0.139836,-0.854153,0.348578,0.157189,-0.881551,766512.66
6252,6253.0,45.0,67.87,3.948,8.684,0.0,2.0,2012.0,9.0,14.0,...,0.099831,-0.541743,-0.039507,-0.057368,-0.049998,-0.011658,-1.053512,0.037501,0.389990,702238.27
6253,6254.0,45.0,65.32,4.038,8.684,0.0,3.0,2012.0,9.0,21.0,...,0.484266,-0.567212,0.023695,-0.327126,-0.244897,0.900082,-0.195598,-0.121139,1.010684,723086.20


In [94]:
train


,id,Store,Temperature,Fuel_Price,Unemployment,IsHoliday,Weekly_Sales,week,year,month,...,Scaled_Promotion2/Scaled_Promotion5,Scaled_Promotion3*Scaled_Promotion4,Scaled_Promotion3+Scaled_Promotion4,Scaled_Promotion3/Scaled_Promotion4,Scaled_Promotion3*Scaled_Promotion5,Scaled_Promotion3+Scaled_Promotion5,Scaled_Promotion3/Scaled_Promotion5,Scaled_Promotion4*Scaled_Promotion5,Scaled_Promotion4+Scaled_Promotion5,Scaled_Promotion4/Scaled_Promotion5
0,1.0,1.0,42.31,2.572,8.106,0.0,1643690.90,1.0,2010.0,5.0,...,0.425376,0.155012,0.809191,0.625560,0.144600,0.775756,0.670602,0.231153,0.962149,1.072003
1,2.0,1.0,38.51,2.548,8.106,1.0,1641957.44,1.0,2010.0,12.0,...,0.425442,0.154199,0.807214,0.624575,0.143675,0.773301,0.670325,0.230037,0.959842,1.073250
2,3.0,1.0,39.93,2.514,8.106,0.0,1611968.17,3.0,2010.0,2.0,...,0.426840,0.140199,0.772507,0.605769,0.127861,0.730168,0.664227,0.211071,0.919825,1.096502
3,4.0,1.0,46.63,2.561,8.106,0.0,1409727.59,4.0,2010.0,2.0,...,0.442564,0.061711,0.541458,0.431082,0.044940,0.438637,0.591948,0.104250,0.653891,1.373168
4,5.0,1.0,46.50,2.625,8.106,0.0,1554806.68,1.0,2010.0,5.0,...,0.429921,0.115409,0.707543,0.563667,0.100147,0.647704,0.649568,0.177670,0.845140,1.152396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251.0,45.0,75.09,3.867,8.684,0.0,734297.87,5.0,2012.0,8.0,...,5.421493,-0.241010,0.915339,-0.189134,0.025703,-0.333890,1.773454,-0.135899,1.008453,-9.376704
6251,6252.0,45.0,75.70,3.911,8.684,1.0,766512.66,2.0,2012.0,7.0,...,1.028246,0.054793,-0.468958,0.889602,0.139836,-0.854153,0.348578,0.157189,-0.881551,0.391836
6252,6253.0,45.0,67.87,3.948,8.684,0.0,702238.27,2.0,2012.0,9.0,...,-0.541743,-0.039507,-0.057368,-1.333263,-0.049998,-0.011658,-1.053512,0.037501,0.389990,0.790175
6253,6254.0,45.0,65.32,4.038,8.684,0.0,723086.20,3.0,2012.0,9.0,...,-0.567212,0.023695,-0.327126,2.021617,-0.244897,0.900082,-0.195598,-0.121139,1.010684,-0.096753


In [96]:
# 9. 전진 선택법https://zephyrus1111.tistory.com/65
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

variables = train1.columns.tolist() ## 설명 변수 리스트

selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0

target = train['Weekly_Sales']
train1 = train.drop(['Weekly_Sales'], axis=1)

while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    for col in remainder: 
        X = sm.add_constant(train[selected_variables+[col]])
        model = sm.OLS(target, X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(target, sm.add_constant(train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break
selected_variables

C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In

KeyboardInterrupt: 

In [95]:
# 11. 단계별 선택법
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

sel_variables = [] ## 선택된 변수들
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0

y = train['Weekly_Sales']
train1 = train.drop(['Weekly_Sales'], axis=1)

while len(list(train1.columns)) > 0:
    cols = list(set(list(train1.columns)) - set(sel_variables))
    pval = pd.Series(index=cols) 
    for col in cols: 
        X = sm.add_constant(train[sel_variables+[col]])
        pval[col] = sm.OLS(target, X).fit().pvalues[col]
 
    if pval.min() < 0.05 : ## 최소 p-value 값이 기준 값보다 작으면 포함
        sel_variables.append(pval.idxmin()) # 최솟값의 인덱스 넣기
        while len(sel_variables) > 0:
            sel_X = sm.add_constant(train[sel_variables])
            sel_pval = sm.OLS(y,sel_X).fit().pvalues[1:] # 절편항의 p-value는 뺀다
            
            if sel_pval.max() >= 0.05: # 최대 p-value값이 기준값보다 크거나 같으면 제외
                sel_variables.remove(sel_pval.idxmax())            
            else:
                break
        
        step += 1
        steps.append(step)
        adjusted_r_squared.append(sm.OLS(y ,sm.add_constant(train[sel_variables])).fit().rsquared_adj)
        sv_per_step.append(sel_variables.copy())
    else:
        break

selected_variables

C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\HOME\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In

KeyboardInterrupt: 

#### 모델링


In [53]:
from xgboost import XGBRegressor, plot_tree, plot_importance
from sklearn.metrics import mean_squared_error

def RMSE(y, y_pred):
    return mean_squared_error(y, y_pred)**0.5

error1 = []
error2 = []
error3 = []
for store in range(1, int(max(train.Store))+1):
    
    train_store = train[train.Store==store]

    train2010 = train_store[(train_store.year.astype('int')==2010) & (train_store.month.astype('int')<=9)]
    train2011 = train_store[(train_store.year.astype('int')==2011) & (train_store.month.astype('int')<=9)]
    train2012 = train_store[(train_store.year.astype('int')==2012) & (train_store.month.astype('int')<9)]
    train_ = pd.concat([train2010, train2011, train2012])
    train_store_2010_2012 = pd.concat([train2010, train2012])
    train_store_2011_2012 = pd.concat([train2011, train2012])
    
    test_ = train_store[(train_store.year.astype('int')==2012) & (train_store.month.astype('int')==9)]
    
    
    y_train1 = train_['Weekly_Sales']
    X_train1 = train_.drop(['Weekly_Sales', 'id' , 'Store', 'Scaled_Promotion1','Scaled_Promotion2', 'Scaled_Promotion3','Scaled_Promotion4', 'Scaled_Promotion5'], axis=1)
    y_test1 = test_['Weekly_Sales']
    X_test1 = test_.drop(['Weekly_Sales', 'id', 'Store', 'Scaled_Promotion1','Scaled_Promotion2', 'Scaled_Promotion3','Scaled_Promotion4', 'Scaled_Promotion5'], axis=1)

    y_train2 = train_store_2010_2012['Weekly_Sales']
    X_train2 = train_store_2010_2012.drop(['Weekly_Sales', 'id', 'Store', 'Scaled_Promotion1','Scaled_Promotion2', 'Scaled_Promotion3','Scaled_Promotion4', 'Scaled_Promotion5'], axis=1)

    y_train3 = train_store_2011_2012['Weekly_Sales']
    X_train3 = train_store_2011_2012.drop(['Weekly_Sales', 'id', 'Store', 'Scaled_Promotion1','Scaled_Promotion2', 'Scaled_Promotion3','Scaled_Promotion4', 'Scaled_Promotion5'], axis=1)
    
    
    
    
    
    # 각각의 모델 학습
    model = XGBRegressor(**{"n_estimators": 60, "min_child_weight": 3, "max_depth": 6})
    model.fit(X_train1, y_train1)
    
    model_2010_2012 = XGBRegressor(**{"n_estimators": 60, "min_child_weight": 3, "max_depth": 6})
    model_2010_2012.fit(X_train2, y_train2)
    
    model_2011_2012 = XGBRegressor(**{"n_estimators": 60, "min_child_weight": 3, "max_depth": 6})
    model_2011_2012.fit(X_train3, y_train3)
    
    # 2012년도 9월에 대해서 예측
    
    pred1 = model.predict(X_test1)
    rmse1 = RMSE(y_test1, pred1)
    pred2 = model_2010_2012.predict(X_test1)
    rmse2 = RMSE(y_test1, pred2)
    pred3 = model_2011_2012.predict(X_test1)
    rmse3 = RMSE(y_test1, pred3)
    
    
    #print('rmse :', rmse1)
    error1.append(rmse1)
    error2.append(rmse2)
    error3.append(rmse3)
    

print('평균 rmse :', np.mean(error1), np.mean(error2), np.mean(error3))

평균 rmse : 97964.92611668224 70289.74109720724 98128.57199701038


In [44]:
import pandas as pd
pd.options.display.max_rows = 60
pd.options.display.max_columns = 20

In [54]:
train

,id,Store,Unemployment,year,Scaled_Promotion1,Scaled_Promotion2,Scaled_Promotion3,Scaled_Promotion4,Scaled_Promotion5,Temperature*Scaled_Promotion1,...,Scaled_Promotion2/Scaled_Promotion5,Scaled_Promotion3*Scaled_Promotion4,Scaled_Promotion3+Scaled_Promotion4,Scaled_Promotion3*Scaled_Promotion5,Scaled_Promotion3+Scaled_Promotion5,Scaled_Promotion3/Scaled_Promotion5,Scaled_Promotion4*Scaled_Promotion5,Scaled_Promotion4+Scaled_Promotion5,month,Weekly_Sales
0,1.0,1.0,8.106,2010.0,0.599813,0.197527,0.311399,0.497792,0.464357,25.378085,...,0.425376,0.155012,0.809191,0.144600,0.775756,0.670602,0.231153,0.962149,5.0,1643690.90
1,2.0,1.0,8.106,2010.0,0.598594,0.196965,0.310337,0.496877,0.462965,23.051861,...,0.425442,0.154199,0.807214,0.143675,0.773301,0.670325,0.230037,0.959842,12.0,1641957.44
2,3.0,1.0,8.106,2010.0,0.578665,0.187273,0.291425,0.481082,0.438743,23.106090,...,0.426840,0.140199,0.772507,0.127861,0.730168,0.664227,0.211071,0.919825,2.0,1611968.17
3,4.0,1.0,8.106,2010.0,0.448005,0.121942,0.163102,0.378356,0.275535,20.890485,...,0.442564,0.061711,0.541458,0.044940,0.438637,0.591948,0.104250,0.653891,2.0,1409727.59
4,5.0,1.0,8.106,2010.0,0.542243,0.168809,0.255053,0.452489,0.392651,25.214319,...,0.429921,0.115409,0.707543,0.100147,0.647704,0.649568,0.177670,0.845140,5.0,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251.0,45.0,8.684,2012.0,3.195845,-0.652682,-0.213502,1.128841,-0.120388,239.975990,...,5.421493,-0.241010,0.915339,0.025703,-0.333890,1.773454,-0.135899,1.008453,8.0,734297.87
6251,6252.0,45.0,8.684,2012.0,0.761857,-0.651263,-0.220780,-0.248178,-0.633373,57.672578,...,1.028246,0.054793,-0.468958,0.139836,-0.854153,0.348578,0.157189,-0.881551,7.0,766512.66
6252,6253.0,45.0,8.684,2012.0,0.835840,-0.118019,-0.229508,0.172140,0.217850,56.728474,...,-0.541743,-0.039507,-0.057368,-0.049998,-0.011658,-1.053512,0.037501,0.389990,9.0,702238.27
6253,6254.0,45.0,8.684,2012.0,0.265630,-0.634680,-0.218864,-0.108262,1.118946,17.350936,...,-0.567212,0.023695,-0.327126,-0.244897,0.900082,-0.195598,-0.121139,1.010684,9.0,723086.20


In [60]:
import eli5 #5살도 이해할수있는 함수
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(model).fit(X_train, y_train)
eli5.show_weights(perm, top = 80, feature_names = X_train.columns.tolist())



Weight,Feature
0.1065 ± 0.0243,Scaled_Promotion2*Scaled_Promotion5
0.0864 ± 0.0119,Scaled_Promotion1+Scaled_Promotion3
0.0688 ± 0.0176,year/Scaled_Promotion1
0.0361 ± 0.0054,Fuel_Price*month
0.0293 ± 0.0102,Scaled_Promotion2/Scaled_Promotion3
0.0250 ± 0.0085,Scaled_Promotion1*Scaled_Promotion3
0.0240 ± 0.0077,month/day
0.0238 ± 0.0092,Scaled_Promotion1*Scaled_Promotion5
0.0159 ± 0.0050,Scaled_Promotion2*Scaled_Promotion4
0.0146 ± 0.0023,id


## AutoML Pycaret 사용하기

In [32]:
from pycaret.regression import *
model = setup(train, target = 'Weekly_Sales', 
              fold_shuffle=True,
              fold=15, 
              session_id=530,
              remove_multicollinearity = True,
              normalize= True,
              n_jobs = -1,
              silent = True,
              combine_rare_levels = True,
              ignore_low_variance = True )

,Description,Value
0,session_id,530
1,Target,Weekly_Sales
2,Original Data,"(6255, 14)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(4378, 25)"


In [33]:
top5_model = compare_models(round=4, n_select = 5, sort='RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0564,0.0057,0.0755,0.9833,0.0052,0.0041,0.4887
xgboost,Extreme Gradient Boosting,0.0563,0.0066,0.0807,0.9809,0.0055,0.0041,0.0747
lightgbm,Light Gradient Boosting Machine,0.0592,0.0073,0.0851,0.9787,0.0058,0.0043,0.0767
et,Extra Trees Regressor,0.0647,0.0104,0.1018,0.9697,0.0069,0.0047,0.1067
rf,Random Forest Regressor,0.0630,0.0113,0.1061,0.9670,0.0071,0.0046,0.1313
dt,Decision Tree Regressor,0.0824,0.0223,0.1479,0.9346,0.0100,0.0060,0.0047
gbr,Gradient Boosting Regressor,0.1627,0.0410,0.2023,0.8806,0.0138,0.0119,0.0427
ada,AdaBoost Regressor,0.3952,0.2075,0.4554,0.3962,0.0311,0.0290,0.0253
knn,K Neighbors Regressor,0.4196,0.2636,0.5131,0.2325,0.0352,0.0309,0.0073
br,Bayesian Ridge,0.4427,0.2808,0.5297,0.1833,0.0363,0.0326,0.0033


In [36]:
# top5_model[:4]
stacker = stack_models(top5_model[:4], optimize = 'RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0500,0.0047,0.0687,0.9865,0.0047,0.0036
1,0.0515,0.0056,0.0745,0.9833,0.0050,0.0037
2,0.0504,0.0042,0.0647,0.9879,0.0044,0.0037
3,0.0487,0.0040,0.0631,0.9890,0.0043,0.0036
4,0.0490,0.0056,0.0750,0.9821,0.0051,0.0036
5,0.0500,0.0042,0.0651,0.9879,0.0044,0.0037
6,0.0534,0.0051,0.0715,0.9856,0.0049,0.0039
7,0.0477,0.0041,0.0640,0.9874,0.0044,0.0035
8,0.0526,0.0052,0.0724,0.9852,0.0049,0.0038


In [41]:
prediction = predict_model(stacker, data=test)
pred = np.expm1(prediction['Label'])

temp = pd.read_csv('data/sample_submission.csv')
temp['Weekly_Sales'] = pred.reset_index()['Label']

submission = pd.DataFrame({
        "id" : temp['id'],
        "Weekly_Sales" : temp['Weekly_Sales']
})
submission.to_csv('submit1.csv', index = False)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,0,0,0,0,0,0
